# Code for Statistical Filtering method

In this file, the codes for our proposed statistical data filtering method are organized by the same order like the one in our paper

## Import needed libraries

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from numpy import std
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
import random
import shutil
import tensorflow.contrib.metrics as metrics
from keras.layers import Flatten, Dense, BatchNormalization, Activation, Input, Dropout, LSTM
from keras.regularizers import l2
from keras.layers.merge import Concatenate
from keras.layers import LeakyReLU
from keras.activations import selu
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop
import random

## Data Wrangling
- This step uses the data Excel file, modifies the headers a little and changes the type of file into .csv --> Please refer to the file DataMarked.csv in this folder
- This section makes the data into 133 features

In [0]:
# Load data into a Data frame
df = pd.read_csv('DayMarked.csv')

# Create an empty array with 133 features with array dimensions follow this structure: (day,hour,features)
data = np.zeros((2975,24,133))

#Set holiday variable at the feature #133 (Boolean: 0 - Not holiday; 1 - Holiday) 
data[:,0,-1] = df.values[:,-2]
data[:,1,-1] = df.values[:,-2]
data[:,2,-1] = df.values[:,-2]
data[:,3,-1] = df.values[:,-2]
data[:,4,-1] = df.values[:,-2]
data[:,5,-1] = df.values[:,-2]
data[:,6,-1] = df.values[:,-2]
data[:,7,-1] = df.values[:,-2]
data[:,8,-1] = df.values[:,-2]
data[:,9,-1] = df.values[:,-2]
data[:,10,-1] = df.values[:,-2]
data[:,11,-1] = df.values[:,-2]
data[:,12,-1] = df.values[:,-2]
data[:,13,-1] = df.values[:,-2]
data[:,14,-1] = df.values[:,-2]
data[:,15,-1] = df.values[:,-2]
data[:,16,-1] = df.values[:,-2]
data[:,17,-1] = df.values[:,-2]
data[:,18,-1] = df.values[:,-2]
data[:,19,-1] = df.values[:,-2]
data[:,20,-1] = df.values[:,-2]
data[:,21,-1] = df.values[:,-2]
data[:,22,-1] = df.values[:,-2]
data[:,23,-1] = df.values[:,-2]

#Set hour variables at features #2 to #25 (Boolean: 0 - Not in the desired hour, 1 - In the desire hour)
data[:,0,1] = 1
data[:,1,2] = 1
data[:,2,3] = 1
data[:,3,4] = 1
data[:,4,5] = 1
data[:,5,6] = 1
data[:,6,7] = 1
data[:,7,8] = 1
data[:,8,9] = 1
data[:,9,10] = 1
data[:,10,11] = 1
data[:,11,12] = 1
data[:,12,13] = 1
data[:,13,14] = 1
data[:,14,15] = 1
data[:,15,16] = 1
data[:,16,17] = 1
data[:,17,18] = 1
data[:,18,19] = 1
data[:,19,20] = 1
data[:,20,21] = 1
data[:,21,22] = 1
data[:,22,23] = 1
data[:,23,24] = 1

#Set hourly values for the corresponding hour at feature #1 (Absolute values of hourly load data)
data[:,0,0] = df.values[:,2]
data[:,1,0] = df.values[:,3]
data[:,2,0] = df.values[:,4]
data[:,3,0] = df.values[:,5]
data[:,4,0] = df.values[:,6]
data[:,5,0] = df.values[:,7]
data[:,6,0] = df.values[:,8]
data[:,7,0] = df.values[:,9]
data[:,8,0] = df.values[:,10]
data[:,9,0] = df.values[:,11]
data[:,10,0] = df.values[:,12]
data[:,11,0] = df.values[:,13]
data[:,12,0] = df.values[:,14]
data[:,13,0] = df.values[:,15]
data[:,14,0] = df.values[:,16]
data[:,15,0] = df.values[:,17]
data[:,16,0] = df.values[:,18]
data[:,17,0] = df.values[:,19]
data[:,18,0] = df.values[:,20]
data[:,19,0] = df.values[:,21]
data[:,20,0] = df.values[:,22]
for i in range(len(df)):
    data[i,21,0] = float(df.values[i,23])
    data[i,22,0] = float(df.values[i,24])
data[:,23,0] = df.values[:,25]

#Set weekday & day-in-a-month & week-in-a-year & month-in-a-year & quarter-in-a-year 
#indices at features #26 to #132 (Boolean: 0 - Not in the desired indicator, 1 - In the desire indicator)

for i in range(len(df)):
    #Weekday indices - Boolean - from feature #26 to #32
    if df.values[i,0] == 'CN':
        data[i,0,31] = 1
        data[i,1,31] = 1
        data[i,2,31] = 1
        data[i,3,31] = 1
        data[i,4,31] = 1
        data[i,5,31] = 1
        data[i,6,31] = 1
        data[i,7,31] = 1
        data[i,8,31] = 1
        data[i,9,31] = 1
        data[i,10,31] = 1
        data[i,11,31] = 1
        data[i,12,31] = 1
        data[i,13,31] = 1
        data[i,14,31] = 1
        data[i,15,31] = 1
        data[i,16,31] = 1
        data[i,17,31] = 1
        data[i,18,31] = 1
        data[i,19,31] = 1
        data[i,20,31] = 1
        data[i,21,31] = 1
        data[i,22,31] = 1
        data[i,23,31] = 1
    else:
        a = int(df.values[i,0])+23
        data[i,0,a] = 1
        data[i,1,a] = 1
        data[i,2,a] = 1
        data[i,3,a] = 1
        data[i,4,a] = 1
        data[i,5,a] = 1
        data[i,6,a] = 1
        data[i,7,a] = 1
        data[i,8,a] = 1
        data[i,9,a] = 1
        data[i,10,a] = 1
        data[i,11,a] = 1
        data[i,12,a] = 1
        data[i,13,a] = 1
        data[i,14,a] = 1
        data[i,15,a] = 1
        data[i,16,a] = 1
        data[i,17,a] = 1
        data[i,18,a] = 1
        data[i,19,a] = 1
        data[i,20,a] = 1
        data[i,21,a] = 1
        data[i,22,a] = 1
        data[i,23,a] = 1
    #Day-in-a-month indices - Boolean - from feature #33 to #63 - 31 days
    b = int(df.values[i,1][-5:-3])+31
    data[i,0,b] = 1
    data[i,1,b] = 1
    data[i,2,b] = 1
    data[i,3,b] = 1
    data[i,4,b] = 1
    data[i,5,b] = 1
    data[i,6,b] = 1
    data[i,7,b] = 1
    data[i,8,b] = 1
    data[i,9,b] = 1
    data[i,10,b] = 1
    data[i,11,b] = 1
    data[i,12,b] = 1
    data[i,13,b] = 1
    data[i,14,b] = 1
    data[i,15,b] = 1
    data[i,16,b] = 1
    data[i,17,b] = 1
    data[i,18,b] = 1
    data[i,19,b] = 1
    data[i,20,b] = 1
    data[i,21,b] = 1
    data[i,22,b] = 1
    data[i,23,b] = 1
    #Week-in-a-year indices - Boolean - from feature #64 to #116 - 53 weeks
    c = int(df.values[i,-1])+62
    data[i,0,c] = 1
    data[i,1,c] = 1
    data[i,2,c] = 1
    data[i,3,c] = 1
    data[i,4,c] = 1
    data[i,5,c] = 1
    data[i,6,c] = 1
    data[i,7,c] = 1
    data[i,8,c] = 1
    data[i,9,c] = 1
    data[i,10,c] = 1
    data[i,11,c] = 1
    data[i,12,c] = 1
    data[i,13,c] = 1
    data[i,14,c] = 1
    data[i,15,c] = 1
    data[i,16,c] = 1
    data[i,17,c] = 1
    data[i,18,c] = 1
    data[i,19,c] = 1
    data[i,20,c] = 1
    data[i,21,c] = 1
    data[i,22,c] = 1
    data[i,23,c] = 1
    #Month-in-a-year indices - Boolean - from feature #117 to #128 - 12 months
    d = int(df.values[i,1][-8:-6])+115
    data[i,0,d] = 1
    data[i,1,d] = 1
    data[i,2,d] = 1
    data[i,3,d] = 1
    data[i,4,d] = 1
    data[i,5,d] = 1
    data[i,6,d] = 1
    data[i,7,d] = 1
    data[i,8,d] = 1
    data[i,9,d] = 1
    data[i,10,d] = 1
    data[i,11,d] = 1
    data[i,12,d] = 1
    data[i,13,d] = 1
    data[i,14,d] = 1
    data[i,15,d] = 1
    data[i,16,d] = 1
    data[i,17,d] = 1
    data[i,18,d] = 1
    data[i,19,d] = 1
    data[i,20,d] = 1
    data[i,21,d] = 1
    data[i,22,d] = 1
    data[i,23,d] = 1
    #Quarter-in-a-year indices - Boolean - from feature #128 to #131 - 4 quarters
    if int(df.values[i,1][-8:-6]) in [1,2,3]:
        data[i,0,128] = 1
        data[i,1,128] = 1
        data[i,2,128] = 1
        data[i,3,128] = 1
        data[i,4,128] = 1
        data[i,5,128] = 1
        data[i,6,128] = 1
        data[i,7,128] = 1
        data[i,8,128] = 1
        data[i,9,128] = 1
        data[i,10,128] = 1
        data[i,11,128] = 1
        data[i,12,128] = 1
        data[i,13,128] = 1
        data[i,14,128] = 1
        data[i,15,128] = 1
        data[i,16,128] = 1
        data[i,17,128] = 1
        data[i,18,128] = 1
        data[i,19,128] = 1
        data[i,20,128] = 1
        data[i,21,128] = 1
        data[i,22,128] = 1
        data[i,23,128] = 1
    elif int(df.values[i,1][-8:-6]) in [4,5,6]:
        data[i,0,129] = 1
        data[i,1,129] = 1
        data[i,2,129] = 1
        data[i,3,129] = 1
        data[i,4,129] = 1
        data[i,5,129] = 1
        data[i,6,129] = 1
        data[i,7,129] = 1
        data[i,8,129] = 1
        data[i,9,129] = 1
        data[i,10,129] = 1
        data[i,11,129] = 1
        data[i,12,129] = 1
        data[i,13,129] = 1
        data[i,14,129] = 1
        data[i,15,129] = 1
        data[i,16,129] = 1
        data[i,17,129] = 1
        data[i,18,129] = 1
        data[i,19,129] = 1
        data[i,20,129] = 1
        data[i,21,129] = 1
        data[i,22,129] = 1
        data[i,23,129] = 1
    elif int(df.values[i,1][-8:-6]) in [7,8,9]:
        data[i,0,130] = 1
        data[i,1,130] = 1
        data[i,2,130] = 1
        data[i,3,130] = 1
        data[i,4,130] = 1
        data[i,5,130] = 1
        data[i,6,130] = 1
        data[i,7,130] = 1
        data[i,8,130] = 1
        data[i,9,130] = 1
        data[i,10,130] = 1
        data[i,11,130] = 1
        data[i,12,130] = 1
        data[i,13,130] = 1
        data[i,14,130] = 1
        data[i,15,130] = 1
        data[i,16,130] = 1
        data[i,17,130] = 1
        data[i,18,130] = 1
        data[i,19,130] = 1
        data[i,20,130] = 1
        data[i,21,130] = 1
        data[i,22,130] = 1
        data[i,23,130] = 1
    else:
        data[i,0,131] = 1
        data[i,1,131] = 1
        data[i,2,131] = 1
        data[i,3,131] = 1
        data[i,4,131] = 1
        data[i,5,131] = 1
        data[i,6,131] = 1
        data[i,7,131] = 1
        data[i,8,131] = 1
        data[i,9,131] = 1
        data[i,10,131] = 1
        data[i,11,131] = 1
        data[i,12,131] = 1
        data[i,13,131] = 1
        data[i,14,131] = 1
        data[i,15,131] = 1
        data[i,16,131] = 1
        data[i,17,131] = 1
        data[i,18,131] = 1
        data[i,19,131] = 1
        data[i,20,131] = 1
        data[i,21,131] = 1
        data[i,22,131] = 1
        data[i,23,131] = 1
        
#Output a zip file that worked with Python .npz
np.save('DataDec23_new.npz',data)

## Data analysis
This section cannot provide everything has been done in the Load data analysis part in our paper, but can only give the functions that are used to provide the insights that has been introduced.
This section guides you through array differencing, PDF calculation function, PCA and Dendrogram calculating and plotting functions with assumptions that we want to plot the PDF of the whole dataset after wrangling and cluster its distribution into separate components as well as group further the same behavior sub-partition of each component with dendrogram.



### Differencing function

In [0]:
# Load data
a = np.load('DataDec23_new.npz')
data_all = a['arr_0']

# Differencing with the scaling function log
df_data_all = pd.Series(data_all[:,:,0].flatten())
ts_log = np.log(df_data_all)
diff = ts_log - ts_log.shift(24)
diff_drop = np.array(diff[27768:]) # the array data is cut off at 27768, 
#which is the position of the time point January 1st, 2014

### PDF calculating and plotting functions

In [0]:
#Plot the PDF function
plt.hist(diff_drop,bins='sqrt',histtype = 'step',color='r')
plt.title("PDF of the Difference Series")
plt.xlabel('Amplitude')
plt.ylabel('Density')
plt.show()
plt.plot(diff_drop)
plt.show()
plt.boxplot(diff_drop)
plt.show()

### PCA function

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) #The number of components defines the dimension of the decomposition matrix, here we have a PCA 2D matrix to indicate the eigen position of each data points 
principalComponents = pca.fit_transform(diff_main[:,:,0].reshape(-1,24))

### Dendrogram

In [0]:
from scipy.cluster import hierarchy
hierarchy.dendrogram(diff_main[:,:,0].reshape(-1,24).T) 

## The main code for proposed statistical data filtering method with confidence level
- To integrate the algorithm to make the code runnable, confidence level must be converted to Sigma values.
- Conversion from confidence levels to Sigma values:
  - 90%: 1.65 sigma
  - 91%: 1.69 sigma
  - 92%: 1.75
  - 93%: 1.81
  - 94%: 1.89
  - 95%: 1.96 sigma
  - 96%: 2.05
  - 97%: 2.17
  - 98%: 2.33
  - 99%: 2.575 sigma
  - 99.73%: 3 sigma
  - 99.9936%: 4 sigma
  - 99.99932%: 4.5 sigma

In [0]:
# calculate summary statistics
data_mean, data_std = mean(data_all[:,:,0].reshape(-1)), std(data_all[:,:,0].reshape(-1))
# identify outliers
cut_off = data_std * 1.65
lower, upper = data_mean - cut_off, data_mean + cut_off

for i in range(data_all.shape[0]):
    for j in range(data_all.shape[1]):
        if data_all[i,j,0] < lower:
            data_all[i,j,0]= lower
        elif data_all[i,j,0] > upper:
            data_all[i,j,0] = upper
print("1")
plt.hist(data_all[:,:,0].reshape(-1),bins='sqrt',histtype = 'step',color='r')
plt.show()

## Test the filtered data with ANN model and print its MAPE

### Sunday

In [0]:
from numpy.random import seed
seed(1611)
from tensorflow import set_random_seed
set_random_seed(3116)

# %% Load data 

a1 = np.load('SUN_to4_95.npz')
data_all1 = a1['arr_0']
te1 = np.load('SUN_to4.npz')
data_te1 = te1['arr_0']
a2 = np.load('SUN_5to6_95.npz')
data_all2 = a2['arr_0']
te2 = np.load('SUN_5to6.npz')
data_te2 = te2['arr_0']
a3 = np.load('SUN_7to15_95.npz')
data_all3 = a3['arr_0']
te3 = np.load('SUN_7to15.npz')
data_te3 = te3['arr_0']
a4 = np.load('SUN_16to23_95.npz')
data_all4 = a4['arr_0']
te4 = np.load('SUN_16to23.npz')
data_te4 = te4['arr_0']

data_all = np.zeros((data_all1.shape[0],24,133))
data_all[:,:5,:] = data_all1
data_all[:,5:7,:] = data_all2
data_all[:,7:16,:] = data_all3
data_all[:,16:,:] = data_all4
data_te = np.zeros((data_te1.shape[0],24,133))
data_te[:,:5,:] = data_te1
data_te[:,5:7,:] = data_te2
data_te[:,7:16,:] = data_te3
data_te[:,16:,:] = data_te4

z = np.load('presun_orig.npz')
orig_test_sun = z['arr_0']
z = np.load('sun_orig.npz')
orig_test_sun_out = z['arr_0']

# %% Split train, test sets
from sklearn.model_selection import train_test_split

inp = data_all[:-int(0.1*len(data_all))-1]
out = data_all[1:-int(0.1*len(data_all))]
test_inp = data_te[-int(0.1*len(data_all)):-1]
test_out = data_te[-int(0.1*len(data_all))+1:]
pretest_inp = orig_test_sun[-int(0.1*len(data_all))+1:]
pretest_out = orig_test_sun_out[-int(0.1*len(data_all))+1:]
x_train, x_val, y_train, y_val = train_test_split(inp,out,
                                                  random_state=42, test_size =0.33,shuffle=False)



# design network
model = Sequential()
model.add(Dense(100, input_shape=(data_all.shape[1],133)))
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(100)) 
model.add(Activation('selu'))
model.add(Flatten())
model.add(Dense(data_all.shape[1]))
rms = RMSprop()
# Compile
model.compile(loss='mae', optimizer='adam', metrics=['mse','mape'])
# fit network
history = model.fit(x_train, y_train[:,:,0], epochs=142, batch_size=4, validation_data=(x_val, y_val[:,:,0]), verbose=1, shuffle=True)

# calculate MAPE
AVG = []
pred = []
plot_val = []
for i in range(len(test_inp)):
    test_pred = model.predict(test_inp[i].reshape(-1,data_all.shape[1],133))
    pred.append(test_pred)
    real_pred = np.log(pretest_inp[i,:,0].T) + test_pred
    real_pred = np.e**real_pred
    print(abs((real_pred.T.reshape(-1)-pretest_out[i,:,0].reshape(-1)))/pretest_out[i,:,0].reshape(-1))
    plot_val.append([real_pred.T.reshape(-1),pretest_out[i,:,0].reshape(-1)])
    AVG.append(np.sum(abs((real_pred.T.reshape(-1)-pretest_out[i,:,0].reshape(-1)))/pretest_out[i,:,0].reshape(-1))*100/24)
AVG = np.array(AVG)
pred=np.array(pred)
print(np.sum(AVG)/len(pretest_inp))

### Monday

In [0]:
from numpy.random import seed
seed(1611)
from tensorflow import set_random_seed
set_random_seed(3116)

# %% Load data 

a1 = np.load('MON_to4_95.npz')
data_all1 = a1['arr_0']
te1 = np.load('MON_to4.npz')
data_te1 = te1['arr_0']
a2 = np.load('MON_5to6_95.npz')
data_all2 = a2['arr_0']
te2 = np.load('MON_5to6.npz')
data_te2 = te2['arr_0']
a3 = np.load('MON_7to16_95.npz')
data_all3 = a3['arr_0']
te3 = np.load('MON_7to16.npz')
data_te3 = te3['arr_0']
a4 = np.load('MON_17to23_95.npz')
data_all4 = a4['arr_0']
te4 = np.load('MON_17to23.npz')
data_te4 = te4['arr_0']

data_all = np.zeros((data_all1.shape[0],24,133))
data_all[:,:5,:] = data_all1
data_all[:,5:7,:] = data_all2
data_all[:,7:17,:] = data_all3
data_all[:,17:,:] = data_all4
data_te = np.zeros((data_te1.shape[0],24,133))
data_te[:,:5,:] = data_te1
data_te[:,5:7,:] = data_te2
data_te[:,7:17,:] = data_te3
data_te[:,17:,:] = data_te4

z = np.load('premon_orig.npz')
orig_test_sun = z['arr_0']
z = np.load('mon_orig.npz')
orig_test_sun_out = z['arr_0']

# %% Split train, test sets
from sklearn.model_selection import train_test_split

inp = data_all[:-int(0.1*len(data_all))-1]
out = data_all[1:-int(0.1*len(data_all))]
test_inp = data_te[-int(0.1*len(data_all)):-1]
test_out = data_te[-int(0.1*len(data_all))+1:]
pretest_inp = orig_test_mon[-int(0.1*len(data_all))+1:]
pretest_out = orig_test_mon_out[-int(0.1*len(data_all))+1:]
x_train, x_val, y_train, y_val = train_test_split(inp,out,
                                                  random_state=42, test_size =0.33,shuffle=False)



# design network
model = Sequential()
model.add(Dense(100, input_shape=(data_all.shape[1],133)))
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(100)) 
model.add(Activation('selu'))
model.add(Flatten())
model.add(Dense(data_all.shape[1]))
rms = RMSprop()
# Compile
model.compile(loss='mae', optimizer='adam', metrics=['mse','mape'])
# fit network
history = model.fit(x_train, y_train[:,:,0], epochs=142, batch_size=4, validation_data=(x_val, y_val[:,:,0]), verbose=1, shuffle=True)

# calculate MAPE
AVG = []
pred = []
plot_val = []
for i in range(len(test_inp)):
    test_pred = model.predict(test_inp[i].reshape(-1,data_all.shape[1],133))
    pred.append(test_pred)
    real_pred = np.log(pretest_inp[i,:,0].T) + test_pred
    real_pred = np.e**real_pred
    print(abs((real_pred.T.reshape(-1)-pretest_out[i,:,0].reshape(-1)))/pretest_out[i,:,0].reshape(-1))
    plot_val.append([real_pred.T.reshape(-1),pretest_out[i,:,0].reshape(-1)])
    AVG.append(np.sum(abs((real_pred.T.reshape(-1)-pretest_out[i,:,0].reshape(-1)))/pretest_out[i,:,0].reshape(-1))*100/24)
AVG = np.array(AVG)
pred=np.array(pred)
print(np.sum(AVG)/len(pretest_inp))

### The rests

In [0]:
from numpy.random import seed
seed(1611)
from tensorflow import set_random_seed
set_random_seed(3116)

# %% Load data


a1 = np.load('RESTs_95.npz')
data_all = a1['arr_0']
te1 = np.load('RESTs.npz')
data_te = te1['arr_0']

r = np.load('rests_orig.npz')
orig_rests = r['arr_0']
orig_test_rests = orig_rests[:-1]
orig_test_rests_out = orig_rests[1:]

# %% Split train, test sets
from sklearn.model_selection import train_test_split

inp = data_all[:-int(0.1*len(data_all))-1]
out = data_all[1:-int(0.1*len(data_all))]
test_inp = data_te[-int(0.1*len(data_all)):-1]
test_out = data_te[-int(0.1*len(data_all))+1:]
pretest_inp = orig_test_rests[-int(0.1*len(data_all))+1:]
pretest_out = orig_test_rests_out[-int(0.1*len(data_all))+1:]
x_train, x_val, y_train, y_val = train_test_split(inp,out,
                                                  random_state=42, test_size =0.33,shuffle=False)



# design network
model = Sequential()
model.add(Dense(100, input_shape=(data_all.shape[1],133)))
model.add(Activation('selu'))
model.add(Dropout(0.5))
model.add(Dense(100)) 
model.add(Activation('selu'))
model.add(Flatten())
model.add(Dense(data_all.shape[1]))
rms = RMSprop()
# Compile
model.compile(loss='mae', optimizer='adam', metrics=['mse','mape'])
# fit network
history = model.fit(x_train, y_train[:,:,0], epochs=142, batch_size=4, validation_data=(x_val, y_val[:,:,0]), verbose=1, shuffle=True)

# calculate MAPE
AVG = []
pred = []
plot_val = []
for i in range(len(test_inp)):
    test_pred = model.predict(test_inp[i].reshape(-1,data_all.shape[1],133))
    pred.append(test_pred)
    real_pred = np.log(pretest_inp[i,:,0].T) + test_pred
    real_pred = np.e**real_pred
    print(abs((real_pred.T.reshape(-1)-pretest_out[i,:,0].reshape(-1)))/pretest_out[i,:,0].reshape(-1))
    plot_val.append([real_pred.T.reshape(-1),pretest_out[i,:,0].reshape(-1)])
    AVG.append(np.sum(abs((real_pred.T.reshape(-1)-pretest_out[i,:,0].reshape(-1)))/pretest_out[i,:,0].reshape(-1))*100/24)
AVG = np.array(AVG)
pred=np.array(pred)
print(np.sum(AVG)/len(pretest_inp))

## Test the filtered data with ARIMA model and print its MAPE

### Sunday

In [0]:
from numpy.random import seed
seed(1611)
from tensorflow import set_random_seed
set_random_seed(3116)

# %% Load data 

a1 = np.load('SUN_to4_95.npz')
data_all1 = a1['arr_0']
te1 = np.load('SUN_to4.npz')
data_te1 = te1['arr_0']
a2 = np.load('SUN_5to6_95.npz')
data_all2 = a2['arr_0']
te2 = np.load('SUN_5to6.npz')
data_te2 = te2['arr_0']
a3 = np.load('SUN_7to15_95.npz')
data_all3 = a3['arr_0']
te3 = np.load('SUN_7to15.npz')
data_te3 = te3['arr_0']
a4 = np.load('SUN_16to23_95.npz')
data_all4 = a4['arr_0']
te4 = np.load('SUN_16to23.npz')
data_te4 = te4['arr_0']

data_all = np.zeros((data_all1.shape[0],24,133))
data_all[:,:5,:] = data_all1
data_all[:,5:7,:] = data_all2
data_all[:,7:16,:] = data_all3
data_all[:,16:,:] = data_all4
data_te = np.zeros((data_te1.shape[0],24,133))
data_te[:,:5,:] = data_te1
data_te[:,5:7,:] = data_te2
data_te[:,7:16,:] = data_te3
data_te[:,16:,:] = data_te4

z = np.load('presun_orig.npz')
orig_test_sun = z['arr_0']
z = np.load('sun_orig.npz')
orig_test_sun_out = z['arr_0']

# %% Split train, test sets
from sklearn.model_selection import train_test_split

inp = data_all[:-int(0.1*len(data_all))-1]
out = data_all[1:-int(0.1*len(data_all))]
test_inp = data_te[-int(0.1*len(data_all)):-1]
test_out = data_te[-int(0.1*len(data_all))+1:]
pretest_inp = orig_test_sun[-int(0.1*len(data_all))+1:]
pretest_out = orig_test_sun_out[-int(0.1*len(data_all))+1:]
x_train, x_val, y_train, y_val = train_test_split(inp,out,
                                                  random_state=42, test_size =0.33,shuffle=False)



from statsmodels.tsa.arima_model import ARIMA
AVG=[]
history = [x for x in data_all[:,:,0].reshape(-1)]
predictions = list()
plot_val1=[]
for t in range(len(pretest_out[:,:,0].reshape(-1))):
  model = ARIMA(history, order=(6,0,0))
  model_fit = model.fit(disp=1)
  output = model_fit.forecast()
  
  
  yhat = output[0]
  test_pred = yhat
  real_pred = np.log(pretest_inp[:,:,0].reshape(-1)[t])+test_pred
  real_pred = np.e**real_pred
  print(test_pred.shape)
  #pred.append(test_pred)
  print(abs((real_pred-pretest_out[:,:,0].reshape(-1)[t]))/pretest_out[:,:,0].reshape(-1)[t])
  plot_val1.append([real_pred,pretest_out[:,:,0].reshape(-1)[t]])
  AVG.append(abs((real_pred-pretest_out[:,:,0].reshape(-1)[t]))/pretest_out[:,:,0].reshape(-1)[t]*100)
AVG = np.array(AVG)
#pred=np.array(pred)
print(np.mean(AVG))

### Monday

In [0]:
from numpy.random import seed
seed(1611)
from tensorflow import set_random_seed
set_random_seed(3116)

# %% Load data 

a1 = np.load('MON_to4_95.npz')
data_all1 = a1['arr_0']
te1 = np.load('MON_to4.npz')
data_te1 = te1['arr_0']
a2 = np.load('MON_5to6_95.npz')
data_all2 = a2['arr_0']
te2 = np.load('MON_5to6.npz')
data_te2 = te2['arr_0']
a3 = np.load('MON_7to16_95.npz')
data_all3 = a3['arr_0']
te3 = np.load('MON_7to16.npz')
data_te3 = te3['arr_0']
a4 = np.load('MON_17to23_95.npz')
data_all4 = a4['arr_0']
te4 = np.load('MON_17to23.npz')
data_te4 = te4['arr_0']

data_all = np.zeros((data_all1.shape[0],24,133))
data_all[:,:5,:] = data_all1
data_all[:,5:7,:] = data_all2
data_all[:,7:17,:] = data_all3
data_all[:,17:,:] = data_all4
data_te = np.zeros((data_te1.shape[0],24,133))
data_te[:,:5,:] = data_te1
data_te[:,5:7,:] = data_te2
data_te[:,7:17,:] = data_te3
data_te[:,17:,:] = data_te4

z = np.load('premon_orig.npz')
orig_test_sun = z['arr_0']
z = np.load('mon_orig.npz')
orig_test_sun_out = z['arr_0']

# %% Split train, test sets
from sklearn.model_selection import train_test_split

inp = data_all[:-int(0.1*len(data_all))-1]
out = data_all[1:-int(0.1*len(data_all))]
test_inp = data_te[-int(0.1*len(data_all)):-1]
test_out = data_te[-int(0.1*len(data_all))+1:]
pretest_inp = orig_test_mon[-int(0.1*len(data_all))+1:]
pretest_out = orig_test_mon_out[-int(0.1*len(data_all))+1:]
x_train, x_val, y_train, y_val = train_test_split(inp,out,
                                                  random_state=42, test_size =0.33,shuffle=False)



from statsmodels.tsa.arima_model import ARIMA
AVG=[]
history = [x for x in data_all[:,:,0].reshape(-1)]
predictions = list()
plot_val1=[]
for t in range(len(pretest_out[:,:,0].reshape(-1))):
  model = ARIMA(history, order=(6,0,0))
  model_fit = model.fit(disp=1)
  output = model_fit.forecast()
  
  
  yhat = output[0]
  test_pred = yhat
  real_pred = np.log(pretest_inp[:,:,0].reshape(-1)[t])+test_pred
  real_pred = np.e**real_pred
  print(test_pred.shape)
  #pred.append(test_pred)
  print(abs((real_pred-pretest_out[:,:,0].reshape(-1)[t]))/pretest_out[:,:,0].reshape(-1)[t])
  plot_val1.append([real_pred,pretest_out[:,:,0].reshape(-1)[t]])
  AVG.append(abs((real_pred-pretest_out[:,:,0].reshape(-1)[t]))/pretest_out[:,:,0].reshape(-1)[t]*100)
AVG = np.array(AVG)
#pred=np.array(pred)
print(np.mean(AVG))

### The rests

In [0]:
from numpy.random import seed
seed(1611)
from tensorflow import set_random_seed
set_random_seed(3116)

# %% Load data


a1 = np.load('RESTs_95.npz')
data_all = a1['arr_0']
te1 = np.load('RESTs.npz')
data_te = te1['arr_0']

r = np.load('rests_orig.npz')
orig_rests = r['arr_0']
orig_test_rests = orig_rests[:-1]
orig_test_rests_out = orig_rests[1:]

# %% Split train, test sets
from sklearn.model_selection import train_test_split

inp = data_all[:-int(0.1*len(data_all))-1]
out = data_all[1:-int(0.1*len(data_all))]
test_inp = data_te[-int(0.1*len(data_all)):-1]
test_out = data_te[-int(0.1*len(data_all))+1:]
pretest_inp = orig_test_rests[-int(0.1*len(data_all))+1:]
pretest_out = orig_test_rests_out[-int(0.1*len(data_all))+1:]
x_train, x_val, y_train, y_val = train_test_split(inp,out,
                                                  random_state=42, test_size =0.33,shuffle=False)



from statsmodels.tsa.arima_model import ARIMA
AVG=[]
history = [x for x in data_all[:,:,0].reshape(-1)]
predictions = list()
plot_val1=[]
for t in range(len(pretest_out[:,:,0].reshape(-1))):
  model = ARIMA(history, order=(6,0,0))
  model_fit = model.fit(disp=1)
  output = model_fit.forecast()
  
  
  yhat = output[0]
  test_pred = yhat
  real_pred = np.log(pretest_inp[:,:,0].reshape(-1)[t])+test_pred
  real_pred = np.e**real_pred
  print(test_pred.shape)
  #pred.append(test_pred)
  print(abs((real_pred-pretest_out[:,:,0].reshape(-1)[t]))/pretest_out[:,:,0].reshape(-1)[t])
  plot_val1.append([real_pred,pretest_out[:,:,0].reshape(-1)[t]])
  AVG.append(abs((real_pred-pretest_out[:,:,0].reshape(-1)[t]))/pretest_out[:,:,0].reshape(-1)[t]*100)
AVG = np.array(AVG)
#pred=np.array(pred)
print(np.mean(AVG))